### In this script we will extract metrics for each commit. For that you need to download and install Understand Tool, available here https://scitools.com/trial-download-3/. You need to install in 'SANER2022-CodeSamples/2-ExtractingMetrics/understand'. To run you will need license to Understand, student get free in https://scitools.com/non-commercial-license/.

In [156]:
pip install GitPython

Note: you may need to restart the kernel to use updated packages.


In [157]:
pip install JPype1

In [158]:
from git import Repo
import pandas as pd
import numpy as np
import os
import fnmatch
from statistics import mean
from concurrent.futures import ThreadPoolExecutor
import shutil
import time

In [159]:
def printStatus(index, size):
    print("{0}% Completed samples".format((index / size) * 100))

In [160]:
def checkoutTo(project, sha):
    projectPath = "repositories/" + project
    repository = Repo(projectPath)
    repository.git.checkout(sha, "-f")

In [161]:
def extractMetricsWithUnderstand(owner, project):
    understandPath = "understand\\SciTools\\bin\\pc-win64\\und"
    os.system('cmd /c "{0} create -languages java {1}"'.format(understandPath, project))
    os.system('cmd /c "{0} add {1} {2}"'.format(understandPath, "repositories/"+owner+"/"+project, project))
    os.system('cmd /c "{0} settings -metrics all {1}"'.format(understandPath, project))
    os.system('cmd /c "{0} settings -metricsOutputFile {1}.csv {1}"'.format(understandPath, project))
    os.system('cmd /c "{0} -quiet analyze {1}"'.format(understandPath, project))
    os.system('cmd /c "{0} metrics {1}"'.format(understandPath, project))

In [162]:
def class_level_metrics(dataframe, owner, project):
    dataframe = dataframe[dataframe["Kind"].str.contains("Class")]
    dataframe["sample"] = owner+"/"+project
    total_lines = dataframe["CountLineCode"].sum()
    total_LOC = dataframe["CountLine"].sum()
    dataframe = dataframe.groupby("sample").mean()
    dataframe["CountTotalLine"] = total_lines
    dataframe["CountTotalLOC"] = total_LOC
    return dataframe

In [163]:
def project_level_metrics(dataframe, owner, project):
    numberJavaFiles = 0 if dataframe.empty else dataframe["Kind"].value_counts()["File"]
    dataframe = dataframe[dataframe["Kind"] == "Package"]
    packages = len(dataframe)
    dataframe.drop(dataframe.columns.difference(['CountDeclClass']), 1, inplace=True)
    dataframe["sample"] = owner+"/"+project
    dataframe = dataframe.groupby("sample").sum()
    dataframe["JavaFiles"] = numberJavaFiles
    dataframe["Packages"] = packages
    return dataframe

In [164]:
def readMetricsFromCsv(project):
    dtype = {'Kind': str, 'Name': str, 'AvgCyclomatic': np.float64, 'AvgCyclomaticModified': np.float64,'AvgCyclomaticStrict': np.float64, 'AvgEssential': np.float64, 'AvgLine': np.float64, 'AvgLineBlank': np.float64,'AvgLineCode': np.float64, 'AvgLineComment': np.float64, 'CountClassBase': np.float64, 'CountClassCoupled': np.float64,'CountClassCoupledModified': np.float64, 'CountClassDerived': np.float64, 'CountDeclClass': np.float64,'CountDeclClassMethod': np.float64, 'CountDeclClassVariable': np.float64,'CountDeclExecutableUnit': np.float64, 'CountDeclFile': np.float64, 'CountDeclFunction': np.float64,'CountDeclInstanceMethod': np.float64, 'CountDeclInstanceVariable': np.float64,'CountDeclMethod': np.float64, 'CountDeclMethodAll': np.float64, 'CountDeclMethodDefault': np.float64,'CountDeclMethodPrivate': np.float64, 'CountDeclMethodProtected': np.float64,'CountDeclMethodPublic': np.float64, 'CountInput': np.float64,'CountLine': np.float64, 'CountLineBlank': np.float64,'CountLineCode': np.float64, 'CountLineCodeDecl': np.float64, 'CountLineCodeExe': np.float64,'CountLineComment': np.float64, 'CountOutput': np.float64, 'CountPath': np.float64, 'CountPathLog': np.float64,'CountSemicolon': np.float64, 'CountStmt': np.float64, 'CountStmtDecl': np.float64, 'CountStmtExe': np.float64,'Cyclomatic': np.float64, 'CyclomaticModified': np.float64, 'CyclomaticStrict': np.float64, 'Essential': np.float64, 'Knots': np.float64, 'MaxCyclomatic': np.float64, 'MaxCyclomaticModified': np.float64,'MaxCyclomaticStrict': np.float64, 'MaxEssential': np.float64,'MaxEssentialKnots': np.float64,'MaxInheritanceTree': np.float64, 'MaxNesting': np.float64, 'MinEssentialKnots': np.float64,'PercentLackOfCohesion': np.float64, 'PercentLackOfCohesionModified': np.float64,'RatioCommentToCode': np.float64, 'SumCyclomatic': np.float64, 'SumCyclomaticModified': np.float64,'SumCyclomaticStrict': np.float64, 'SumEssential': np.float64, 'CountTotalLine': np.float64, 'CountTotalLOC': np.float64}
    dataframe = pd.read_csv(project+".csv", dtype=dtype)
    return dataframe

In [165]:
def getUnderstandMetrics(owner, project):
    extractMetricsWithUnderstand(owner, project)
    
    dataframe = readMetricsFromCsv(project)
        
    class_level = class_level_metrics(dataframe.copy(), owner, project)
    class_level["level"] = "class"
    class_level.set_index("level")    
    
    project_level = project_level_metrics(dataframe.copy(), owner, project)
    project_level["level"] = "project"
    project_level.set_index("level")
    
    return pd.concat([class_level, project_level], axis=0)

In [166]:
def findPaths(pattern, path):
    result = []
    for root, dirs, files in os.walk(path):
        if '.git' in root:
            continue
        for name in files:
            if fnmatch.fnmatch(name, pattern):
                result.append(os.path.join(root, name))
    return result

In [167]:
def getMetrics(owner, project):
    metrics = getUnderstandMetrics(owner, project)
    return metrics

In [168]:
def deleteUnusedFiles(sample):
    os.remove(sample + ".udb")
    os.remove(sample + ".csv")    

In [169]:
def createDirectoryIfNotExists(dirName):
    if not os.path.exists(dirName):
            os.makedirs(dirName)

In [170]:
def extractMetricsByCommit(sample):
    owner, project = sample.split("/")
    metrics = getMetrics(owner, project)
    return metrics

In [171]:
def clone(gitUrl, repoDir, sample):
    Repo.clone_from(gitUrl, repoDir + sample)

In [172]:
def download(sample):
    gitHubUrl = createGitHubUrl(sample)
    print("Downloading %s" % sample)
    repoDir = "repositories/"
    isdir = os.path.isdir(repoDir+sample)
    if isdir:
        print("Project " + sample + " previously downloaded")
    else:
        clone(gitHubUrl, repoDir, sample)
        print("%s downloaded" % sample)

In [173]:
def createGitHubUrl(sample):
    return "https://github.com/" + sample + ".git"

In [174]:
createDirectoryIfNotExists("repositories")
createDirectoryIfNotExists("metrics")

In [175]:
exclude = []

In [176]:
samples = pd.read_csv("../5-RepoFilter/output.csv", index_col=0)

In [177]:
lista = samples["watcher_repo"].tolist()
for i in range(len(lista)):
    sample = lista[i]
    if sample == "jkYishon/android6.0":
        print(i)

40
448
648


In [178]:
# samples = samples[40:]

In [179]:
def run_as_thread(input_row):
    repository = input_row["watcher_repo"]
    try:
        download(repository)
    except:
        return

    owner = repository.split("/")[0]
    repo = repository.split("/")[1]

    createDirectoryIfNotExists(f"repositories/{owner}")
    createDirectoryIfNotExists(f"metrics/{owner}")

    metrics = extractMetricsByCommit(repository)

    metrics.to_csv("metrics/"+repository+".csv", index=False)

    deleteUnusedFiles(f"{repo}")

In [180]:
limit = 5
executor = ThreadPoolExecutor(3)
for index in samples.index:
    while(executor._work_queue.qsize() > limit):
        print("waiting... " + str(index))
        time.sleep(1)
    sample = samples.loc[index]
    future = executor.submit(run_as_thread, (sample))
    print(f"{index} was Dropped")
    print(f"Progresso => {(index/samples.index[-1]) * 100}")
executor.shutdown(wait=True, cancel_futures=False)

21534 was DroppedProject xiangjiaowangzi/TransitionDemo previously downloaded

Progresso => 5.7468135528085575
23797 was DroppedProject yuliskov/MyVideoTube previously downloaded

Progresso => 6.350744038087919
Progresso => 6.35154465296014
37953 was Dropped
Progresso => 10.128578748478832
38182 was Dropped

Progresso => 10.189692350391768
38208 was Dropped
Progresso => 10.19663101261769
39099 was Dropped
Progresso => 10.434413629667585
39706 was Dropped
Progresso => 10.596404705480476
40584 was Dropped
Progresso => 10.830717991417409
waiting... 40867
Project yuliskov/SmartTubeNext previously downloaded
waiting... 40867
waiting... 40867
waiting... 40867
waiting... 40867
waiting... 40867
waiting... 40867
waiting... 40867
waiting... 40867
waiting... 40867
waiting... 40867
waiting... 40867
waiting... 40867
waiting... 40867
waiting... 40867
waiting... 40867


C:\Users\mrgab\AppData\Local\Temp/ipykernel_11432/3338126027.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe["sample"] = owner+"/"+project
C:\Users\mrgab\AppData\Local\Temp/ipykernel_11432/379348769.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  dataframe.drop(dataframe.columns.difference(['CountDeclClass']), 1, inplace=True)
C:\Users\mrgab\AppData\Local\Temp/ipykernel_11432/379348769.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html

Project morristech/ModuleLoader previously downloaded
40867 was Dropped
Progresso => 10.906242661030339
waiting... 40951
waiting... 40951
waiting... 40951
Project tun-lin/HappyShop previously downloaded
40951 was Dropped
Progresso => 10.92865987745255
waiting... 41048
waiting... 41048
waiting... 41048
waiting... 41048
waiting... 41048
waiting... 41048
waiting... 41048
waiting... 41048
waiting... 41048
waiting... 41048
waiting... 41048
Project mananwason/CountSteps previously downloaded
41048 was Dropped
Progresso => 10.954546424987724
waiting... 41052
waiting... 41052
waiting... 41052
waiting... 41052
Project ddwhan0123/ReactNativeDemo previously downloaded
41052 was Dropped
Progresso => 10.95561391148402
waiting... 41059
waiting... 41059
waiting... 41059
waiting... 41059
waiting... 41059
waiting... 41059
waiting... 41059
Project androidmalin/android-architecture-todomvp previously downloaded
41059 was Dropped
Progresso => 10.957482012852537
waiting... 41064
waiting... 41064
waiting...

71686 was Dropped
Progresso => 19.130959243365574
waiting... 80043
waiting... 80043
waiting... 80043
waiting... 80043
Project 00zhengfu00/prj previously downloaded
80043 was Dropped
Progresso => 21.36120540575162
waiting... 80391
waiting... 80391
waiting... 80391
waiting... 80391
waiting... 80391
waiting... 80391
waiting... 80391
waiting... 80391
Project msisuzney/tv-waterfall-layout previously downloaded
80391 was Dropped
Progresso => 21.45407673092935
waiting... 80878
Project jimmonster/aac_common previously downloaded
80878 was Dropped
Progresso => 21.58404321185337
waiting... 81166
waiting... 81166
waiting... 81166
waiting... 81166
waiting... 81166
waiting... 81166
waiting... 81166
waiting... 81166
waiting... 81166
waiting... 81166
waiting... 81166
waiting... 81166
waiting... 81166
waiting... 81166
waiting... 81166
Project Lucas1Jorge/App_Usage_Tracker previously downloaded
81166 was Dropped
Progresso => 21.66090223958667
waiting... 81173
waiting... 81173
waiting... 81173
waiting..

waiting... 89018
waiting... 89018
waiting... 89018
waiting... 89018
waiting... 89018
waiting... 89018
waiting... 89018
waiting... 89018
waiting... 89018
waiting... 89018
waiting... 89018
waiting... 89018
waiting... 89018
waiting... 89018
waiting... 89018
waiting... 89018
waiting... 89018
waiting... 89018
waiting... 89018
waiting... 89018
waiting... 89018
waiting... 89018
waiting... 89018
waiting... 89018
waiting... 89018
waiting... 89018
waiting... 89018
waiting... 89018
waiting... 89018
waiting... 89018
waiting... 89018
waiting... 89018
waiting... 89018
waiting... 89018
waiting... 89018
waiting... 89018
waiting... 89018
waiting... 89018
waiting... 89018
waiting... 89018
waiting... 89018
waiting... 89018
waiting... 89018
waiting... 89018
waiting... 89018
waiting... 89018
waiting... 89018
waiting... 89018
waiting... 89018
waiting... 89018
waiting... 89018
waiting... 89018
waiting... 89018
waiting... 89018
waiting... 89018
waiting... 89018
waiting... 89018
waiting... 89018
waiting... 890

...

waiting... 309940
waiting... 309940
waiting... 309940
waiting... 309940
waiting... 309940
waiting... 309940
waiting... 309940
waiting... 309940
waiting... 309940
waiting... 309940
waiting... 309940
waiting... 309940
waiting... 309940
Project andresgarbio/Smokio previously downloaded
309940 was Dropped
Progresso => 82.71419116548175
waiting... 310210
waiting... 310210
waiting... 310210
waiting... 310210
waiting... 310210
waiting... 310210
waiting... 310210
waiting... 310210
waiting... 310210
waiting... 310210
Project andresgarbio/Smokio-user previously downloaded
310210 was Dropped
Progresso => 82.78624650398172
waiting... 312018
waiting... 312018
waiting... 312018
waiting... 312018
waiting... 312018
waiting... 312018
waiting... 312018
waiting... 312018
waiting... 312018
waiting... 312018
Project morristech/ModuleLoader previously downloaded
312018 was Dropped
Progresso => 83.26875040030744
waiting... 312151
waiting... 312151
waiting... 312151
waiting... 312151
waiting... 312151
waiting

waiting... 324502
waiting... 324502
waiting... 324502
waiting... 324502
waiting... 324502
waiting... 324502
Project Coder-DanielHan/AndroidTrainingSample previously downloaded
324502 was Dropped
Progresso => 86.60037575524669
waiting... 324978
waiting... 324978
waiting... 324978
waiting... 324978
waiting... 324978
waiting... 324978
waiting... 324978
waiting... 324978
waiting... 324978
waiting... 324978
waiting... 324978
waiting... 324978
Project shuijilove/ZXingCodeDemo previously downloaded
324978 was Dropped
Progresso => 86.72740664830589
waiting... 325417
waiting... 325417
waiting... 325417
waiting... 325417
waiting... 325417
waiting... 325417
waiting... 325417
waiting... 325417
waiting... 325417
waiting... 325417
waiting... 325417
Project wangrling/approjects previously downloaded
325417 was Dropped
Progresso => 86.84456329127437
waiting... 326140
waiting... 326140
waiting... 326140
waiting... 326140
waiting... 326140
waiting... 326140
waiting... 326140
waiting... 326140
waiting...

waiting... 329658
waiting... 329658
waiting... 329658
waiting... 329658
waiting... 329658
waiting... 329658
waiting... 329658
waiting... 329658
waiting... 329658
waiting... 329658
waiting... 329658
waiting... 329658
waiting... 329658
waiting... 329658
waiting... 329658
Project Ritesh-Kumar-4946/cairodock previously downloaded
329658 was Dropped
Progresso => 87.97636584897202
waiting... 330364
waiting... 330364
waiting... 330364
waiting... 330364
waiting... 330364
waiting... 330364
waiting... 330364
waiting... 330364
waiting... 330364
waiting... 330364
waiting... 330364
waiting... 330364
waiting... 330364
waiting... 330364
waiting... 330364
waiting... 330364
waiting... 330364
Project subinkrishna/SlidingTabsBasicSample previously downloaded
330364 was Dropped
Progresso => 88.16477721556822
waiting... 330496
waiting... 330496
waiting... 330496
waiting... 330496
waiting... 330496
waiting... 330496
waiting... 330496
waiting... 330496
waiting... 330496
waiting... 330496
waiting... 330496
Pr

342378 was Dropped
Progresso => 91.37097290719272
waiting... 342617
waiting... 342617
waiting... 342617
waiting... 342617
waiting... 342617
waiting... 342617
waiting... 342617
waiting... 342617
waiting... 342617
waiting... 342617
waiting... 342617
waiting... 342617
waiting... 342617
waiting... 342617
waiting... 342617
waiting... 342617
waiting... 342617
Project yangjae/android-ui previously downloaded
342617 was Dropped
Progresso => 91.4347552253464
waiting... 343154
waiting... 343154
waiting... 343154
waiting... 343154
waiting... 343154
waiting... 343154
waiting... 343154
waiting... 343154
waiting... 343154
waiting... 343154
waiting... 343154
waiting... 343154
waiting... 343154
waiting... 343154
waiting... 343154
waiting... 343154
waiting... 343154
waiting... 343154
waiting... 343154
waiting... 343154
waiting... 343154
waiting... 343154
waiting... 343154
waiting... 343154
waiting... 343154
waiting... 343154
waiting... 343154
waiting... 343154
waiting... 343154
waiting... 343154
waitin

waiting... 362712
waiting... 362712
waiting... 362712
waiting... 362712
waiting... 362712
waiting... 362712
waiting... 362712
waiting... 362712
Project ckwcc/App-Architecture previously downloaded
362712 was Dropped
Progresso => 96.79754051111253
waiting... 370485
waiting... 370485
waiting... 370485
Project ckwcc/PictureInPicture previously downloaded
370485 was Dropped
Progresso => 98.87193364503939
waiting... 370674
waiting... 370674
waiting... 370674
waiting... 370674
waiting... 370674
waiting... 370674
waiting... 370674
waiting... 370674
waiting... 370674
waiting... 370674
Project zhangwenhaojf40/GoogleSample previously downloaded
370674 was Dropped
Progresso => 98.92237238198936
waiting... 370783
waiting... 370783
waiting... 370783
waiting... 370783
waiting... 370783
waiting... 370783
waiting... 370783
waiting... 370783
waiting... 370783
waiting... 370783
waiting... 370783
Project KobeBryant824/MaterialDesignWidget previously downloaded
370783 was Dropped
Progresso => 98.951461389